In [ ]:
#01_cleandata.ipynb
#this script is used to conduct initial data cleanings
#created: 07-12-2024
#author: Alessia Kettlitz

In [3]:
#importing libraries
import pandas as pd, numpy as np, matplotlib as plt, seaborn as sns

from pyprojroot.here import here #importing here function for relative file paths

from joblib import dump, load #for saving dataframes

In [4]:
#importing data
data_raw = pd.read_csv(here('raw_data/odp_idb_2001_2022_ddg_compliant.csv'))